In [1]:
%%bash
pip install --upgrade pip
pip install  --disable-pip-version-check \
  torch==1.13.1 \
  torchdata==0.5.1 --quiet

pip install transformers==4.27.2 --quiet
pip install datasets==2.11.0 --quiet
pip install evaluate==0.4.0 --quiet
pip install rouge_score==0.1.2 --quiet
pip install loralib==0.1.1 --quiet
pip install peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.7 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, \
TrainingArguments, GenerationConfig, Trainer
from datasets import load_dataset
import time
import evaluate
import torch
import pandas as pd
# lora lib
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
# data processing
data_name = 'knkarthick/dialogsum'
model_name = 'google/flan-t5-small'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')


dataset = load_dataset(data_name)

print(dataset)

model= AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

# tokenizer function
def tokenizer_function(example):
  start_prompt = 'Summarize the following conversation'
  end_prompt = '\n\nSummary: '
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]

  example['input_ids'] = tokenizer(
      prompt, padding='max_length', truncation=True, return_tensors='pt'
      ).input_ids
  example['labels'] = tokenizer(
      example['summary'], padding='max_length', truncation=True
      # return_tensors='pt'
      ).input_ids

  return example

tokenized_datasets = dataset.map(tokenizer_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(
    ['id', 'topic', 'dialogue', 'summary']
    )

# print the dataset shape
print(f'Shape of the datasets:')
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Shape of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)


In [4]:
lora_config = LoraConfig(
    r=32, # rank matrix
    lora_alpha=32,
    target_modules=['q', 'v'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

# add Lora adapter layer/parametes to the orignal LLM to be trained
peft_model = get_peft_model(
    model,
    lora_config
)
print(peft_model.print_trainable_parameters())

trainable params: 1376256 || all params: 78337408 || trainable%: 1.7568311680672406
None


In [5]:
# train the model
  # full fine tuning
model_dir = 'model_finetuned'
training_arg = TrainingArguments(
    output_dir = model_dir,
    auto_find_batch_size=True,
    overwrite_output_dir = True,
    num_train_epochs = 2,
    learning_rate = 1e-3,
    logging_steps=100,
    weight_decay=0.01
)

trainer = Trainer(
    model = peft_model,
    args = training_arg,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation']
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,9.840500
200,2.531900
300,2.143300
400,1.998300
500,1.929700
600,1.884400
700,1.851600
800,1.829900
900,1.816700
1000,1.797100


TrainOutput(global_step=3116, training_loss=2.0727632501771116, metrics={'train_runtime': 1971.207, 'train_samples_per_second': 12.642, 'train_steps_per_second': 1.581, 'total_flos': 4737749609349120.0, 'train_loss': 2.0727632501771116, 'epoch': 2.0})

In [8]:
# Compare the orignial model summaries & Fine-tuned model summaries
from tqdm import tqdm

instruct_model = trainer.model.to('cpu')

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# dialogues = dataset['test']['dialogue']
# human_baseline_summaries = dataset['test']['summary']

original_model_summaries = []
trained_model_summaries = []

pbar = tqdm(total=len(dialogues))
for _, dialogue in enumerate(dialogues):
  prompt = f"""
SUmmarize the following conversation.
{dialogue}
Summary: """

  input_ids = tokenizer(prompt, return_tensors='pt').input_ids
  original_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
  original_model_summaries.append(original_model_text_output)

  trained_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  trained_model_text_output = tokenizer.decode(trained_model_outputs[0], skip_special_tokens=True)
  trained_model_summaries.append(trained_model_text_output)
  pbar.update(1)
pbar.close()

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, trained_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['huam_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df.head()

100%|██████████| 10/10 [00:43<00:00,  4.33s/it]


,huam_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1# asks #Person1# to take a dictation f...,#Person1# asks #Person2# to take a dictation f...
1,In order to prevent employees from wasting tim...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks #Person1# to take a dictation f...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# is asked to take a dictation for #Pe...,#Person1# asks #Person1# to take a dictation f...
3,#Person2# arrives late because of traffic jam....,#Person1# is stuck in traffic again and #Perso...,#Person1# thinks #Person1# is stuck in traffic...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1# tells #Person1# #Person1# got stuck ...,#Person1#'s got stuck in traffic again. #Perso...


In [9]:
# evaluate the summaries using Rogue
import evaluate

rouge = evaluate.load('rouge')

test = df.loc[2:4, :]
original_model_summaries = test['original_model_summaries'].to_list()
human_baseline_summaries = test['huam_baseline_summaries'].to_list()
trained_model_summaries = test['instruct_model_summaries'].to_list()

original_model_result = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True
)

instruction_model_result = rouge.compute(
    predictions=trained_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True
)

print('Original Model')
print(original_model_result)
print('Instruction Tuned')
print(instruction_model_result)

Original Model
{'rouge1': 0.29509277889559576, 'rouge2': 0.07666537101819461, 'rougeL': 0.20475072658171248, 'rougeLsum': 0.20475072658171248}
Instruction Tuned
{'rouge1': 0.32130718954248366, 'rouge2': 0.11428766909062477, 'rougeL': 0.23372549019607844, 'rougeLsum': 0.23372549019607844}
